In [ ]:
import os
os.chdir("/Users/zbutler/research/fire_prediction")
from geometry.grid_conversion import get_latlon_xy_fxns, ak_bb
from geometry.get_xys import append_xy
from util.daymonth import day2monthday, monthday2day
from prediction.fire_clustering import cluster_fires
from metrics.fire_metrics import get_summary_stats, get_time_series
import cPickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
%matplotlib inline

In [ ]:
# First load our bigass dataframe
with open("data/ak_fires.pkl") as fpkl:
    modis = cPickle.load(fpkl)

In [ ]:
# Now, make our nice data frame
print "%d modis fires" %(len(modis))
ak_fires = append_xy(modis, ak_bb)
print "%d ak fires" %(len(ak_fires))
with open('data/ak_fires.pkl','w') as fpkl:
    cPickle.dump(ak_fires, fpkl, cPickle.HIGHEST_PROTOCOL)
ak_fires_2015 = ak_fires[ak_fires.year == 2015]
print "%d ak 2015 fires" %(len(ak_fires_2015))

In [ ]:
# Try several clustering thresholds
threshs = [1, 2, 5, 10, 20, 50, 100, 200]
n_clusts = []
for thresh in threshs:
    n_fires, fires = cluster_fires(ak_fires_2015, thresh)
    n_clusts.append(n_fires)
    print "Done with %d" %thresh
plt.scatter(threshs, n_clusts, c='r')
plt.show()
plt.close()

In [ ]:
import metrics.fire_metrics as fm
reload(fm)
thresh = 5
n_clusts, clusts = cluster_fires(ak_fires_2015, thresh)
print "%d fires" %(n_clusts)
summary_stat_dict = fm.get_summary_stats(ak_fires_2015, clusts, n_clusts)
time_series = fm.get_time_series(ak_fires_2015, clusts, n_clusts, zero_centered=False)
time_series_0_centered = fm.get_time_series(ak_fires_2015, clusts, n_clusts, zero_centered=True)

In [ ]:
# Plots!
plt.hist(summary_stat_dict['len'], bins=100)
plt.title("histogram of fire sizes")
plt.show()

plt.hist(np.log(summary_stat_dict['len'] + 1), bins=100)
plt.title("Log histogram of fire sizes")
plt.show()

plt.hist(summary_stat_dict['area'], bins=100)  # convert to km
plt.title("Histogram of convex hull of fires")
plt.show()

plt.hist(summary_stat_dict['dist_from_center'], bins=100) # convert to km
plt.title("Histogram of mean distance from cluster center (in km)")
plt.show()

plt.hist(summary_stat_dict['duration'], bins=100)
plt.title("Histograms of fire duration")
plt.show()

In [ ]:
# Time series
for clust,series in enumerate(time_series):
    plt.plot(series, np.zeros(len(series)) + clust, 'r.')
plt.title("Time series (not zero centered)")
plt.show()
plt.close()

for clust,series in enumerate(time_series_0_centered):
    plt.plot(series, np.zeros(len(series)) + clust, 'r.')
plt.title("Time series (zero centered)")
plt.show()
plt.close()

In [ ]:
day2monthday(220)

In [ ]:
boop = ak_fires[np.in1d(ak_fires.year, [2013,2015])]
print len(boop)
print len(ak_fires)
print len(ak_fires_2013)

In [ ]:
import geometry
reload(geometry)
from carpentry.get_feat_df import get_feat_df
get_feat_df(2013, 'data/feat_df.pkl')

In [ ]:
with open("data/feat_df_2013.pkl") as fpkl:
    feat_df = cPickle.load(fpkl)
print "len feat df: " + str(len(feat_df))

In [ ]:
feat_df

In [ ]:
n_big_fires = 0
for fireid in xrange(162):
    if np.max(feat_df[feat_df.fire_id == fireid].n_det_cum) > 4:
        n_big_fires += 1
print "%d big fires" % n_big_fires

In [ ]:
# Lets plot each fire's trajectory overlayed on to one plot
n_fires = np.max(feat_df.fire_id)
for fire in xrange(n_fires):
    det_arr = feat_df[feat_df.fire_id == fire].n_det
    if len(det_arr) < 5:
        continue
    plt.figure(figsize=(20,10))
    plt.plot(np.arange(len(det_arr)), det_arr, 'r-')
    plt.show()
    plt.close()

In [ ]:
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(7,14))
n_clusts, clusts = cluster_fires(ak_fires_2015, 20)

centers = np.zeros((n_clusts,2))
for fire in xrange(n_clusts):
    clust_fires = ak_fires_2015[clusts == fire]
    centers[fire,0] = np.mean(clust_fires.lat)
    centers[fire,1] = np.mean(clust_fires.long)

mp = Basemap(projection="merc",
                 lat_0=ak_bb[0], lon_0=ak_bb[2],
                 llcrnrlat=ak_bb[0],
                 llcrnrlon=ak_bb[2],
                 urcrnrlat=ak_bb[1],
                 urcrnrlon=ak_bb[3],
                 resolution='i')
center_lons, center_lats = mp(centers[:,1], centers[:,0])
mp.plot(center_lons, center_lats, 'rs')
mp.drawmapboundary(fill_color='aqua')
mp.fillcontinents(color='yellow',lake_color='aqua')
mp.drawcoastlines()
plt.show()